### Import packages

In [ ]:
import getpass
from azure.storage.blob import BlobServiceClient
import xarray as xr
import os
import fsspec
from stactools.ecmwf_forecast import stac

### Install locally

In [ ]:
pip install -e ./ecmwf-forecast

### Set up storage account connection string

In [ ]:
connection_string =  getpass.getpass('<storage-account-connection-string>')
container = 'ecmwf-new'
account_name = 'lawistorage'
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
blob_container_client = blob_service_client.get_container_client(container)

### Create STAC item

In [ ]:
filepath = 'https://lawistorage.blob.core.windows.net/ecmwf-new/20220121/00z/0p4-beta/wave/20220121000000-0h-wave-fc.grib2'
item = stac.create_item([filepath])

### Load dataset via kerchunk indices

In [ ]:
%%time

m = fsspec.get_mapper('reference://', fo=item.properties['kerchunk_indices'], remote_options={"connection_string": connection_string})
ds = xr.open_dataset(m, engine='zarr')
ds['swh'].mean()

### Load dataset via original .GRIB2 file

In [ ]:
%%time

import urllib.request
import xarray as xr

url = 'https://ai4edataeuwest.blob.core.windows.net/ecmwf/20220121/00z/0p4-beta/wave/20220121000000-0h-wave-fc.grib2'
filename, _ = urllib.request.urlretrieve(url)

ds = xr.open_dataset(filename, engine="cfgrib")
ds['swh'].mean()

In [ ]:
2.63/0.099